# Two-layer model parameters

Replicate AR6 Chapter 7 samples based on
notebooks/140_WG3_draw_fair_samples.ipynb
in https://github.com/IPCC-WG1/Chapter-7

In [25]:
import sys
import json
import numpy as np
import scipy.stats as stats
import pandas as pd
from netCDF4 import Dataset

In [2]:
sys.path.append('../..')

In [3]:
from mce.util.io import RetrieveGitHub, write_nc

In [19]:
ddir = '../../data/ar6-ch7'
owner = 'IPCC-WG1'
repo = 'Chapter-7'
repo_ch7 = RetrieveGitHub(owner, repo, ddir)

In [9]:
path = repo_ch7.retrieve('data_input/random_seeds.json')
with path.open() as f1:
    SEEDS = json.load(f1)

In [6]:
NINETY_TO_ONESIGMA = stats.norm.ppf(0.95)

SAMPLES = 1000000
F2XCO2_MEAN = 4.00
F2XCO2_NINETY = 0.48

In [10]:
path = repo_ch7.retrieve('data_input/tunings/cmip6_twolayer_tuning_params.json')
with path.open() as f1:
    params = json.load(f1)

cmip6_models = list(params['q4x']['model_data']['EBM-epsilon'].keys())

In [11]:
df_parms = pd.concat({
    vn: pd.DataFrame(params[vn]['model_data']).rename_axis('Dataset')
    for vn in params
}, names=['Parameter']).rename_axis(columns='Model').unstack('Parameter')

In [15]:
pnames = ['q4x', 'lamg', 'cmix', 'cdeep', 'gamma_2l', 'eff']
df1 = df_parms['EBM-epsilon'].loc[cmip6_models, pnames]

kernel = stats.gaussian_kde(df1.T)
df = pd.DataFrame(
    kernel.resample(size=int(SAMPLES*1.25), seed=SEEDS[15]),
    index=df1.columns,
).T

df['lamg'] = 1.
df = df.where(df > 0)
df['lamg'] = stats.truncnorm.rvs(
    -2, 2, loc=-4/3, scale=0.5, size=int(SAMPLES*1.25),
    random_state=SEEDS[16],
)

df_sample = (
    df
    .dropna()
    .iloc[:SAMPLES]
    .reset_index(drop=True)
    .rename_axis(index='Member')
    .rename_axis(columns='')
)

In [16]:
df_sample.head()

,q4x,lamg,cmix,cdeep,gamma_2l,eff
Member,,,,,,
0,8.142751,-1.156137,9.493752,56.066688,0.455056,1.344132
1,6.119064,-2.167808,8.747261,52.676899,0.461327,1.077195
2,8.617112,-1.293838,8.009019,123.943687,0.566571,1.153560
3,7.946020,-1.305245,7.816008,19.885165,0.608541,1.879368
4,8.254456,-0.987906,8.534034,27.923307,0.584929,1.258868


In [17]:
f2x = stats.norm.rvs(
    loc=F2XCO2_MEAN, scale=F2XCO2_NINETY/NINETY_TO_ONESIGMA,
    size=SAMPLES, random_state=SEEDS[73],
)
df = df_sample
ecs = -f2x / df.loc[:, 'lamg']
tcr = f2x / (-df.loc[:, 'lamg'] + df.loc[:, 'gamma_2l'] * df.loc[:, 'eff'])

In [18]:
dims = ('member',)
var_dict = {'member': (np.arange(SAMPLES, dtype='i8'), dims, {})}

var_dict.update({f'geoff__{pn}': (d1.values, dims, {}) for pn, d1 in df.iteritems()})

var_dict['f2x'] = (f2x, dims, {})
var_dict['ecs'] = (ecs.values, dims, {})
var_dict['tcr'] = (tcr.values, dims, {})

In [20]:
path = f'{ddir}/fair_samples_thermal.nc'
write_nc(path, var_dict, {}, dim_unlimited='member')

## Chack results

In [23]:
dir_ch7 = '../../../../../work/Chapter-7-main'

In [26]:
ncf = Dataset(path)

In [27]:
np.allclose(
    pd.DataFrame({
        pn[7:]: ncv[:] for pn, ncv in ncf.variables.items()
        if pn.startswith('geoff__')
    }).sub(
        pd.read_csv(f'{dir_ch7}/data_output_large/geoff_sample.csv', index_col=0)
    ),
    0.,
)

True

In [28]:
[
    np.allclose(
        ncf.variables[pn][:],
        np.load(f'{dir_ch7}/data_input_large/fair-samples/{pn}_unconstrained.npy'),
    )
    for pn in ['f2x', 'ecs', 'tcr']
]

[True, True, True]

In [29]:
ncf.close()